In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

In [2]:
# RDD만을 이용한 데이터 추출

In [3]:

movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [4]:
# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [5]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
print(movie_att.take(3))
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))), (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))), (1, (('어벤져스', '마블'), (13934592, 'KR')))]


[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [6]:
# CASE 2. filter 먼저, join 나중에 -> 더 유리
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [7]:
sc.stop()

In [8]:
# Spark SQL 사용해 보기

In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [30]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [31]:
#스키마를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

In [32]:
# 2. 데이터 프레임 만들기

In [33]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [34]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [35]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [36]:
df.createOrReplaceTempView("movies")

In [37]:
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [38]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [39]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [40]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [41]:
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [42]:
# 년,월,일 
df.select("year", "month", "day").show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2012|    4| 26|
|2013|    6| 13|
|2008|    8|  6|
|2014|    1| 16|
|2008|    4| 30|
+----+-----+---+



In [43]:
# 2013 이후 영화
df.filter(df.year >= 2013).show()


+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [44]:
# 마블, DC 같이 꺼내기  
df.filter((df.company == "마블") | (df.company == "DC")).show()


+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [53]:
df.createOrReplaceTempView('movies')

## SQL 실습 

In [46]:
# 영화 이름만 가져오기 

query = """
SELECT name
    FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [49]:
# 2010년 이후에 개봉한 영화를 조회

query = """
SELECT *
FROM movies
WHERE year >= 2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [50]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """
SELECT name, company
FROM movies
WHERE year < 2012
"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [51]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """
SELECT *
FROM movies 
WHERE name LIKE '%맨'
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [52]:

# BETWEEN 특정 데이터와 데이터 사이를 조회

# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8

query = """
SELECT *
FROM movies 
WHERE month BETWEEN 4 AND 8
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# Join 구현하기

In [56]:

attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [57]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [58]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [59]:

att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [60]:
att_df.createOrReplaceTempView("att")

In [67]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [68]:
# df와 join
query = """
SELECT movies.id, movies.name, movies.company, att.att
FROM movies 
JOIN att ON movies.id = att.id
"""
spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



In [69]:
spark.stop()

In [ ]:
# 데이터 프레임 API

In [ ]:
# select
df.select("*").collect()

In [ ]:
df.select("name", "company").collect()

In [ ]:
df.select(df.name, (df.year-2000).alias("year")).show()

In [ ]:
# agg : Aggreagte의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id": "count"}).collect()

In [ ]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

In [ ]:
df.groupBy().avg().collect()

In [ ]:
# 회사별 개봉월의 평균
df.groupBy('company').agg({"month": "mean"}).collect()

In [ ]:
# 회사 별 월 별 영화 개수 정보


In [ ]:
# join : 다른 데이터 프레임과 사용자가 지정한 컬럼을 기준으로 합치는 작업
df.join(att_df, 'id').select(df.name, att_df.att).show()

In [ ]:
# select, where, orderBy 절 사용
marvel_df = df.select("name", "company", "year").where("company=='마블'").orderBy("id")
marvel_df.collect()

In [ ]:
spark.stop()
sc.stop()

In [ ]:
# SQL 최적화

In [70]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [73]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [74]:
# inferSchema : 자동으로 스키마 예측하게 하기
data = spark.read.csv(trip_file, inferSchema=True, header=True)

In [75]:
data.createOrReplaceTempView("mobility_data")

In [76]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [ ]:
# 스파크 SQL을 사용하는 이유

In [77]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [78]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#614, 'count(1) AS trips#615]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#541 as string),  , -1)[0]], [split(cast(pickup_datetime#541 as string),  , -1)[0] AS pickup_date#614, count(1) AS trips#615L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#539,dispatching_base_num#540,pickup_datetime#541,dropoff_datetime#542,PULocationID#543,DOLocationID#544,SR_Flag#545])
      +- Relation [hvfhs_license_num#539,dispatching_base_num#540,pickup_datetime#541,dropoff_datetime#542,PULocationID#543,DOLocationID#544,SR_Flag#545] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#619], [_groupingexpression#619 AS pickup_date#614, count(1) AS trips#615L]
+- Project [split(cast(pickup_datetime#541 as string),  , -1)[0] AS _groupingexpression#619]
   +- Rel

In [79]:
# 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#623]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#622]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#622], [pickup_date#622, count(1) AS trips#623L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#541 as string),  , -1)[0] AS pickup_date#622]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#539,dispatching_base_num#540,pickup_datetime#541,dropoff_datetime#542,PULocationID#543,DOLocationID#544,SR_Flag#545])
            +- Relation [hvfhs_license_num#539,dispatching_base_num#540,pickup_datetime#541,dropoff_datetime#542,PULocationID#543,DOLocationID#544,SR_Flag#545] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#622], [pickup_date#622, count(1) AS tri

In [80]:
spark.stop()

In [81]:
sc.stop()

In [82]:
# 운행 데이터 프레임 생성, Zone 데이터 프레임 생성

In [83]:
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [84]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [86]:
#운행 데이터 프레임 생성, Zone 데이터프레임 생성
trip_data = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/fhvhv_tripdata_2020-03.csv')
zone_data = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/taxi+_zone_lookup.csv')

In [87]:
trip_data.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [88]:
zone_data.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [ ]:
## 승차 Location(PULocationID)별 개수 세기
# 하차 Location(DOLocationID)별 개수 세기
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
#운송사별 운행 건수 비교
#승차 위치 Borough별 운행 건수
#서비스 존별 승차/하차 건수

In [90]:
# csv 컬럼, 내용 확인 
print('trip data column: ', trip_data.columns)
print('zone data column: ', zone_data.columns)
print(trip_data.show(3))
print(zone_data.show(3))

trip data column:  ['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag']
zone data column:  ['LocationID', 'Borough', 'Zone', 'service_zone']
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
+-----------------+--------------------+-------------------+-------------------+------------+-

In [93]:
#승차 Location(PULocationID)별 개수 세기
from pyspark.sql.functions import col, count

trip_data.groupBy('PULocationID')\
    .agg(count('*').alias('pickup_count'))\
    .orderBy(col('PULocationID'))\
    .show(10)

+------------+------------+
|PULocationID|pickup_count|
+------------+------------+
|           1|         362|
|           2|          23|
|           3|       34076|
|           4|       49145|
|           5|        3189|
|           6|        6283|
|           7|      125458|
|           8|         297|
|           9|       10448|
|          10|       44290|
+------------+------------+
only showing top 10 rows



In [94]:
# 하차 Location(DOLocationID)별 개수 세기
trip_data.groupBy('DOLocationID')\
    .agg(count('*').alias('pickup_count'))\
    .orderBy(col('DOLocationID'))\
    .show(10)

+------------+------------+
|DOLocationID|pickup_count|
+------------+------------+
|           1|       65066|
|           2|          21|
|           3|       32728|
|           4|       41781|
|           5|        3126|
|           6|        6541|
|           7|      124255|
|           8|         340|
|           9|       10554|
|          10|       36941|
+------------+------------+
only showing top 10 rows



In [97]:
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고,
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
trip_data.filter(col('hvfhs_license_num') == 'HV0003')\
        .groupBy('PULocationID')\
        .agg(count('*').alias('trip_count'))\
        .orderBy(col('trip_count').desc())\
        .show(10)

+------------+----------+
|PULocationID|trip_count|
+------------+----------+
|          61|    163091|
|          76|    134198|
|         132|    114179|
|          79|    112017|
|          37|    110150|
|          42|    108070|
|         138|    104119|
|         244|     97324|
|          89|     95724|
|          39|     94484|
+------------+----------+
only showing top 10 rows



In [99]:
#운송사별 운행 건수 비교
trip_data.groupBy('hvfhs_license_num')\
        .agg(count('*').alias('trip_count_per_comp'))\
        .orderBy(col('trip_count_per_comp').desc())\
        .show(10)

+-----------------+-------------------+
|hvfhs_license_num|trip_count_per_comp|
+-----------------+-------------------+
|           HV0003|            9836763|
|           HV0005|            3219535|
|           HV0004|             336606|
+-----------------+-------------------+



In [111]:
# 승차 위치 Borough별 운행 건수
pickup_zone = trip_data.join(zone_data, trip_data.PULocationID == zone_data.LocationID, 'left')

pickup_zone.groupBy('Borough')\
            .agg(count('*').alias('trip_count'))\
            .orderBy(col('trip_count').desc())\
            .show()


+-------------+----------+
|      Borough|trip_count|
+-------------+----------+
|    Manhattan|   4953140|
|     Brooklyn|   3735764|
|       Queens|   2437383|
|        Bronx|   2086592|
|Staten Island|    178818|
|      Unknown|       845|
|          EWR|       362|
+-------------+----------+



In [112]:
# 서비스 존별 승차/하차 건수

# 승차 건수
trip_data.join(zone_data, trip_data.PULocationID == zone_data.LocationID, "left") \
    .groupBy("service_zone") \
    .agg(count("*").alias("pickup_count")) \
    .orderBy(col("pickup_count").desc()) \
    .show()

+------------+------------+
|service_zone|pickup_count|
+------------+------------+
|   Boro Zone|     9046897|
| Yellow Zone|     4025190|
|    Airports|      319610|
|         N/A|         845|
|         EWR|         362|
+------------+------------+



In [114]:
# 하자 건수 
trip_data.join(zone_data, trip_data.DOLocationID == zone_data.LocationID, 'left')\
    .groupBy("service_zone")\
    .agg(count('*').alias('pickup_count'))\
    .orderBy(col('pickup_count').desc())\
    .show(10)



+------------+------------+
|service_zone|pickup_count|
+------------+------------+
|   Boro Zone|     8885136|
| Yellow Zone|     3643787|
|    Airports|      411156|
|         N/A|      387759|
|         EWR|       65066|
+------------+------------+



In [115]:
spark.stop()